# 步骤一、加载环境

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
import torch
import pandas as pd
import re
#set_seed(42)

# 步骤二、加载模型

In [2]:
#model_7b_path = '/root/share/model_repos/internlm2-chat-7b'
#model_7b_path = '/root/project/bisai2/sft-guanfang-self/merged_7b_e10'
model_7b_path = '/root/model/internlm2-chat-7b-wendongmi'
tokenizer_7b = AutoTokenizer.from_pretrained(model_7b_path, trust_remote_code=True)
model_7b = AutoModelForCausalLM.from_pretrained(model_7b_path, device_map="auto", trust_remote_code=True, load_in_4bit=True)
model_7b = model_7b.eval()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
response, history = model_7b.chat(tokenizer_7b, "请你做一个自我介绍", history=[])
response

/root/.conda/envs/internlm-demo/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


'您好，我是一款基于人工智能技术的智能助手，致力于通过语言模型为用户提供服务。 '

In [3]:
model_20b_path = '/root/share/model_repos/internlm2-chat-20b'
tokenizer_20b = AutoTokenizer.from_pretrained(model_20b_path, trust_remote_code=True)
model_20b = AutoModelForCausalLM.from_pretrained(model_20b_path, device_map="auto", trust_remote_code=True, load_in_4bit=True)
model_20b = model_20b.eval()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/21 [00:00<?, ?it/s]

# 步骤三、初始化工具类函数

In [4]:
# 抽取字符串
def extract_xuanxiang(text):
    # 给定的字符串
    
    # 使用正则表达式匹配"正确选项是："和"。"之间的内容
    pattern = r"正确选项是：([^。]+)"
    match = re.search(pattern, text)
    
    # 如果找到匹配项，提取并打印内容
    if match:
        extracted_content = match.group(1)
        return extracted_content
    else:
        print("没有找到匹配的内容")
        return ''


# 步骤四、做多选题

In [5]:
#选择题评测
#设置
do_sample = False
temperature = 0.1 
top_p = 0.3
meta_instruction = '你是上市公司的董秘，你乐于助人，诚实无害，你竭诚为投资者解答关于公司运营、财务状况、投资者关系等方面的问题。'

import pandas as pd

#df = pd.DataFrame(pd.read_csv('name.csv',header=1))
# 读取xls文件
df = pd.read_excel('./data/gong_gao_ce_ping_you_xuan_xiang.xls')

df.loc[:, '大模型答案'] = ""
df.loc[:, '大模型回复'] = ""
for index, row in df.iterrows():
    #if index > 2:
    #    break
    
    print(index, end="")
    prompt = f"""{row['评测问题']}

请你围绕问题题干并忠实于公告原文，从下面的选项中筛选出正确的选项，你的回复里面只保留正确的选项：    
选项A：{row['选项A']}
选项B：{row['选项B']}
选项C：{row['选项C']}
选项D：{row['选项D']}
"""
    
    response1, history = model_20b.chat(tokenizer_20b, prompt, history=[],do_sample=do_sample,temperature=temperature,top_p=top_p,meta_instruction=meta_instruction)
    response1, history = model_20b.chat(tokenizer_20b, prompt, history=[],do_sample=do_sample,meta_instruction=meta_instruction)

    #answer = extract_xuanxiang(response)
    #print(response1)
    print("---")
    #print(f"response1: {response1}")
    prompt = f"""{response1}
    从以上多项选择题的题解中提取出正确的答案选项，不要对选项进行分析，答案用“、”分隔，正确的选项是：
"""
    answer_list = []
    response2, history = model_20b.chat(tokenizer_20b, prompt, history=[],do_sample=do_sample,meta_instruction=meta_instruction)
    #print(f"response2: {response2}")
    if 'A' in response2:
        answer_list.append("A")
    if 'B' in response2:
        answer_list.append("B")
    if 'C' in response2:
        answer_list.append("C")
    if 'D' in response2:
        answer_list.append("D")

    answer = "、".join(answer_list)
    print(f"answer: {answer}")
    #response = "da mo xing hui da" + str(index)
    df.at[index, '大模型答案'] = answer
    df.at[index, '大模型回复'] = response1
    
filename = '/root/project/bisai2/sft-guanfang/gong_gao_ce_ping_you_xuan_xiang_多选_result_.csv'

# 保存数据到CSV文件
df.to_csv(filename, index=False)
 
print('CSV文件保存成功！')

0

/root/.conda/envs/internlm-demo/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/.conda/envs/internlm-demo/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/root/.conda/envs/internlm-demo/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.floa

---
answer: C
1

/root/.conda/envs/internlm-demo/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/.conda/envs/internlm-demo/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


---
answer: A
2---
answer: A、B
3---
answer: A、C
4---
answer: A
5---
answer: A、B、C
6---
answer: B
7---
answer: A、C、D
8---
answer: A、B、C、D
9---
answer: A、B
10---
answer: 
11---
answer: A、B、C、D
12---
answer: 
13---
answer: B
14---
answer: B
15---
answer: D
16---
answer: A
17---
answer: A、B、C、D
18---
answer: A
19---
answer: C
20---
answer: A、B、C、D
21---
answer: C、D
22---
answer: A、B、C、D
23---
answer: A
24---
answer: C
25---
answer: C
26---
answer: C
27---
answer: C
28---
answer: B、D
29---
answer: B、C
30---
answer: A、B、C、D
31---
answer: C
32---
answer: A、B、C
33---
answer: B、C
34---
answer: A、C、D
35---
answer: B
36---
answer: D
37---
answer: A、B、C、D
38---
answer: A、C、D
39---
answer: A、B、C、D
40---
answer: A、C
41---
answer: A、C、D
42---
answer: B
43---
answer: A、B、D
44---
answer: C
45---
answer: A、D
46---
answer: A、B、C、D
47---
answer: A、B、C、D
48---
answer: D
49---
answer: A、C
50---
answer: A
51---
answer: D
52---
answer: A、B
53---
answer: A
54---
answer: A、B、D
55---
answer: B
56---
answer: A、C


In [ ]:
0---
response1: 根据公告内容，我们可以分析出以下信息：

1. **沈晓苏先生因到龄退休，已递交辞职申请，辞去公司第八届监事会主席、监事职务。**
2. **张小龙先生被推举为公司第八届监事会召集人。**

现在，让我们根据这些信息来分析每个选项：

**选项A - “体现了公司对年轻化领导力的倾向，有助于引入新思维和管理模式”**：
这个选项提出了一个可能的积极影响，即引入新的领导力和管理模式。然而，公告中并没有明确提到年轻化或新思维的引入，因此我们不能确定这是否是公司决策的意图。

**选项B - “可能会引起公司内部的权力重组和人事变动，影响短期内的稳定性”**：
这个选项提出了一个可能的负面影响，即人事变动可能带来的短期不稳定。虽然公告中提到了人事变动（沈晓苏先生的退休和张小龙先生的推举），但并没有足够的信息来确定这将如何影响公司的短期稳定性。

**选项C - “显示了公司对资深员工的尊重和感激，有利于提高员工的归属感和忠诚度”**：
这个选项提出了一个积极的方面，即公司对退休员工的尊重和感激可能提高员工的归属感和忠诚度。公告中确实提到了对沈晓苏先生贡献的肯定和感谢，这可能对员工产生积极影响。

**选项D - “可能会导致监事会与董事会之间权力平衡的变化，影响决策效率”**：
这个选项提出了一个可能的负面影响，即权力平衡的变化可能影响决策效率。公告中没有提及任何关于权力平衡变化的信息，因此我们无法确定这是否是一个实际的问题。

综上所述，选项C“显示了公司对资深员工的尊重和感激，有利于提高员工的归属感和忠诚度”是最符合公告内容的分析，因为它直接与公告中提到的对沈晓苏先生贡献的肯定和感谢相呼应。因此，正确答案是C。 
response2: C 
answer: C
1---
response1: 根据公告内容，我们可以分析出以下信息：

1. **股权激励计划设计合理，能够明确激励对象，提高激励效率**：公告中明确了股权激励计划是根据相关法律法规和股东大会授权进行的，且授予对象是符合授予条件的28名激励对象。这表明激励计划的设计是符合规定的，并且授予对象是经过筛选的。

2. **授予限制性股票的数量可能过多，导致公司股权过度稀释**：公告中提到授予了546.5万股限制性股票，这的确是一个较大的数量。然而，公告中并未提供公司的总股本信息，因此无法直接判断这个数量是否会导致股权过度稀释。

3. **激励对象的选择标准不明确，可能引起内部不公平和矛盾**：公告中并未详细说明激励对象的选择标准，只是提到授予了28名符合条件的激励对象。这可能使得一些员工对选择标准产生疑问，从而引发不公平感。

4. **限制性股票授予的条件和解锁标准设置得当，有利于激励对象长期贡献**：公告中提到限制性股票的授予和解锁条件是由公司董事会决定的，但具体条件并未在公告中明确列出。因此，我们无法判断这些条件是否得当。

综上所述，选项A“股权激励计划设计合理，能够明确激励对象，提高激励效率”是正确的选项，因为公告中明确了股权激励计划是根据相关法律法规和股东大会授权进行的，且授予对象是符合授予条件的28名激励对象。其他选项要么缺乏足够信息（如B和D），要么与公告内容不符（如C）。因此，正确答案是A。 
response2: A、股权激励计划设计合理，能够明确激励对象，提高激励效率 
answer: A
2---
response1: 根据公告内容，我们可以分析出回购股份用于员工持股计划或股权激励计划对公司内部管理的潜在影响。

选项A：可以提升员工的归属感和忠诚度，增加他们对公司未来发展的信心。
选项B：股权激励可以激发员工的工作热情和创新能力，提高公司整体的竞争力。
选项C：长期持有要求可能导致员工对股价波动过度敏感，影响其工作表现。
选项D：实施股权激励计划可能会增加公司的管理成本和复杂性。

结合公告内容，我们可以分析出：

- 选项A和B都是正面的影响。回购股份用于员工持股计划或股权激励计划可以提升员工的归属感和忠诚度，增加他们对公司未来发展的信心，同时股权激励可以激发员工的工作热情和创新能力，提高公司整体的竞争力。
- 选项C是一个潜在的中性或负面影响。长期持有要求可能导致员工对股价波动过度敏感，这可能会影响其工作表现，但公告中并未提及长期持有要求，因此这个选项可能并不适用。
- 选项D是一个潜在的负面影响。实施股权激励计划可能会增加公司的管理成本和复杂性，但公告中并未提及具体的管理成本和复杂性，因此这个选项可能并不适用。

综上所述，根据公告内容，选项A和B是正确的选项，因为它们描述了回购股份用于员工持股计划或股权激励计划的正面影响。选项C和D虽然可能存在，但根据公告内容，它们并不是最直接和明确的影响。因此，正确选项是A和B。 
response2: A、B 
answer: A、B
CSV文件保存成功！

In [11]:
#选择题评测
#提示词按   问题  选项   要求来组织
#效果不好  不能严格按格式来回复  temperature: float = 0.8, top_p: float = 0.8, meta_instruction:

import pandas as pd

temperature = 0.1 
top_p = 0.8
meta_instruction = '你是上市公司的董秘，你乐于助人，诚实无害，你竭诚为投资者解答关于公司运营、财务状况、投资者关系等方面的问题。'

#df = pd.DataFrame(pd.read_csv('name.csv',header=1))
# 读取xls文件
df = pd.read_excel('./data/gong_gao_ce_ping_you_xuan_xiang.xls')

df.loc[:, '大模型答案'] = ""
df.loc[:, '大模型回复'] = ""
for index, row in df.iterrows():
    print(index, end="")
    prompt = f"""{row['评测问题']}
选项A：{row['选项A']}
选项B：{row['选项B']}
选项C：{row['选项C']}
选项D：{row['选项D']}


请你从上面的选项中选择正确的选项，正确的选项可能不止一个，你要围绕问题题干并忠实于公告原文进行分析
按以下格式组织回答：
1、假如答案只有C，按格式：“正确选项是：C。”给出答案。假如答案A、B、C都对，则格式“正确选项是：A、B、C。”给出答案
2、按格式：“A:4,B:3,C:9,D:6”对每个选项进行把握度打分，分数从0到10，分数越高，表示把握越大，不要分析，只给结果
3、从“与题干的相关程度、原文对选项的支持程度、选项的合理性”等方面对每个选项进行分析总结
"""
    
    response, history = model_20b.chat(tokenizer_20b, prompt, history=[])
    answer = extract_xuanxiang(response)
    print(f"response1: {response1}")
    print(f"answer: {answer}")
    #response = "da mo xing hui da" + str(index)
    df.at[index, '大模型答案'] = answer
    df.at[index, '大模型回复'] = response
    
filename = '/root/project/bisai2/sft-guanfang/gong_gao_ce_ping_you_xuan_xiang_多选_result_.csv'

# 保存数据到CSV文件
df.to_csv(filename, index=False)
 
print('CSV文件保存成功！')

0response1: A：可以提升员工的归属感和忠诚度，增加他们对公司未来发展的信心。
B：股权激励可以激发员工的工作热情和创新能力，提高公司整体的竞争力。
D：实施股权激励计划可能会增加公司的管理成本和复杂性。 
answer: C
1response1: A：可以提升员工的归属感和忠诚度，增加他们对公司未来发展的信心。
B：股权激励可以激发员工的工作热情和创新能力，提高公司整体的竞争力。
D：实施股权激励计划可能会增加公司的管理成本和复杂性。 
answer: A、D
2response1: A：可以提升员工的归属感和忠诚度，增加他们对公司未来发展的信心。
B：股权激励可以激发员工的工作热情和创新能力，提高公司整体的竞争力。
D：实施股权激励计划可能会增加公司的管理成本和复杂性。 
answer: A、B、D
3response1: A：可以提升员工的归属感和忠诚度，增加他们对公司未来发展的信心。
B：股权激励可以激发员工的工作热情和创新能力，提高公司整体的竞争力。
D：实施股权激励计划可能会增加公司的管理成本和复杂性。 
answer: A、B、C、D
4response1: A：可以提升员工的归属感和忠诚度，增加他们对公司未来发展的信心。
B：股权激励可以激发员工的工作热情和创新能力，提高公司整体的竞争力。
D：实施股权激励计划可能会增加公司的管理成本和复杂性。 
answer: A、B、D
5

KeyboardInterrupt: 

In [ ]:
#选择题评测

import pandas as pd

#df = pd.DataFrame(pd.read_csv('name.csv',header=1))
# 读取xls文件
df = pd.read_excel('./data/gong_gao_ce_ping_you_xuan_xiang.xls')

df.loc[:, '大模型答案'] = ""
df.loc[:, '大模型回复'] = ""
for index, row in df.iterrows():
    print(index, end="")
    prompt = f"""{row['评测问题']}

请你围绕问题题干并忠实于公告原文，从下面的选项中筛选出正确的选项，你的回复里面只保留正确的选项：    
选项A：{row['选项A']}
选项B：{row['选项B']}
选项C：{row['选项C']}
选项D：{row['选项D']}
"""
    
    response1, history = model_20.chat(tokenizer20, prompt, history=[])
    #answer = extract_xuanxiang(response)
    #print(response1)
    print("---")
    prompt = f"""{response1}
    从以上多项选择题的题解中提取出正确的答案选项，不要对选项进行分析，答案用“、”分隔，正确的选项是：
"""
    answer_list = []
    response2, history = model20.chat(tokenizer20, prompt, history=[])
    print(response2)
    if 'A' in response2:
        answer_list.append("A")
    if 'B' in response2:
        answer_list.append("B")
    if 'C' in response2:
        answer_list.append("C")
    if 'D' in response2:
        answer_list.append("D")

    answer = "、".join(answer_list)
    print("---+")
    print(answer)
    print("---+++")
    #response = "da mo xing hui da" + str(index)
    df.at[index, '大模型答案'] = answer
    df.at[index, '大模型回复'] = response1
    
filename = '/root/project/bisai2/sft-guanfang/gong_gao_ce_ping_you_xuan_xiang_多选_result_.csv'

# 保存数据到CSV文件
df.to_csv(filename, index=False)
 
print('CSV文件保存成功！')